In [26]:
import pandas as pd
import geopandas as gpd
import folium as fl
import uuid
import pycountry

In [27]:
# In order to only focus on the pipelines part of the GEM European tracker, we will need to filter out the pipelines from the GEM Global tracker that are not in Europe.

european_pipeline_names = pd.read_excel('GEM-data/Europe-Gas-Tracker-2024-01.xlsx', sheet_name='Gas pipelines - data')['PipelineName'].unique()

In [28]:
# Read the GEM-data/GEM-GGIT-Gas-Pipelines-2023-12.geojson file
gem_global_pipelines = gpd.read_file('GEM-data/GEM-GGIT-Gas-Pipelines-2023-12.geojson') 

# Only include pipelines that are in Europe, as per comment in the previous cell
gem_global_pipelines = gem_global_pipelines[gem_global_pipelines['PipelineName'].isin(european_pipeline_names)]

# Only include pipelines that are operating or mothballed
gem_global_pipelines = gem_global_pipelines[gem_global_pipelines['Status'].isin(['Operating', 'Mothballed'])]

# Filter out the pipelines that are relevant to add to our network
relevant_pipelines = [
    'Trans-Mediterranean Gas Pipeline',
    'Greenstream Gas Pipeline',
    'Medgaz Gas Pipeline',
    'Maghreb-Europe Gas Pipeline',
    'Kohtla-Järve-Leningrad Gas Pipeline',
    'Minsk-Kaliningrad Interconnection Gas Pipeline',
    'Valdai-Pskov-Riga Gas Pipeline',
    'Torzhok-Minsk-Ivatsevichy Gas Pipeline',
    'Yamal-Europe Gas Pipeline',
    'Torzhok-Smolensk-Mazyr-Dolyna Gas Pipeline',
    'Urengoy-Pomary-Uzhgorod Gas Pipeline',
    'Progress Gas Pipeline',
    'Yelets-Kremenchuk–Kryvyi Rih Gas Pipeline',
    'Urengoy-Novopskov Gas Pipeline',
    'Petrovsk-Novopskov Gas Pipeline',
    'Orenburg-Novopskov Gas Pipeline',
    'Shebelinka-Dnipropetrovsk–Kryvyi Rih–Rozdilna-Izmail Gas Pipeline',
    'Rozdilna-Izmail Gas Pipeline',
    'TurkStream Gas Pipeline',
    'Turk 2 Stream Gas Pipeline',
    'Blue Stream Gas Pipeline',
    'Kyiv–Western Border Pipeline',
    'Kyiv–Western Border of Ukraine Gas Pipeline',
    "Kremenchuk-Anan'iv-Bohorodchany Gas Pipeline",
    'Ananjiv-Bohorodchani Gas Pipeline',
    "Anan'iv-Tiraspol-Izmail Gas Pipeline",
    'Tula-Shostka-Kiev Gas Pipeline',
    'Yelets-Kursk-Kyiv Gas Pipeline',
    'Yelets-Kursk-Dykanka Gas Pipeline',
    'Soyuz Gas Pipeline',
    'Shebelinka-Poltava-Kiev Gas Pipeline',
    'Shebelinka-Dikanka-Kiev Gas Pipeline',
    'Gryazovets-Volkhov-Slavyanskaya Gas Pipeline',
    'Gryazovets-Vyborg Gas Pipeline',
    'Trans-Anatolian Gas Pipeline',
    'Sarikali-Karacabey Gas Pipeline',
    'Trans-Balkan Gas Pipeline',
    'Murati-Silivri Pipeline',
    'Turkey-Greece-Italy Interconnector Gas Pipeline (ITGI)',
    'Tekirdag-Buyukkaristiran Pipeline',
    'Ivatsevichy-Kobryn-Dolyna Gas Pipeline'
]

# Split the gem_global_pipelines into two dataframes, one with the excluded pipelines and one with the included pipelines
relevant_pipelines_df = gem_global_pipelines[gem_global_pipelines['PipelineName'].isin(relevant_pipelines)]
non_relevant_pipelines_df = gem_global_pipelines[~gem_global_pipelines['PipelineName'].isin(relevant_pipelines)]

# Read the Scigrid_data/IGGIN_PipeSegments.geojson file
scigrid_pipelines_df = gpd.read_file('Scigrid_data/IGGIN_PipeSegments.geojson')

In [29]:
# Plot both the relevant, non-relevant and scigrid pipelines on a map
m = fl.Map(location=[50, 10], zoom_start=4)

# Generate a unique color for each relevant pipeline
colors = ['#FF0000', '#00FF00', '#0000FF', '#FFFF00', '#00FFFF', '#FF00FF', '#800000', '#008000', '#000080', '#808000', '#800080', '#008080', '#808080', '#C0C0C0', '#FFA500', '#800000', '#FF00FF', '#808000', '#800080', '#008080', '#808080', '#C0C0C0', '#FFA500', '#800000', '#FF00FF', '#808000', '#800080', '#008080', '#808080', '#C0C0C0', '#FFA500']

# Add the relevant pipelines layer to the map with unique colors
for idx, pipeline in enumerate(relevant_pipelines):
    color = colors[idx % len(colors)]  # Use modulo operator to cycle through the colors list
    fl.GeoJson(relevant_pipelines_df[relevant_pipelines_df['PipelineName'] == pipeline],
               name=f'Relevant Pipeline - {pipeline}',
               style_function=lambda feature, color=color: {'color': color}).add_to(m)

# Add the non-relevant pipelines layer to the map with a blue color
fl.GeoJson(non_relevant_pipelines_df, name='Non-Relevant Pipelines', style_function=lambda feature: {'color': 'blue'}).add_to(m)

# Add the scigrid pipelines layer to the map with a green color
fl.GeoJson(scigrid_pipelines_df, name='SciGRID Pipelines', style_function=lambda feature: {'color': 'green'}).add_to(m)

# Add layer control to the map
fl.LayerControl().add_to(m)

m.save('relevant_pipelines.html')

## Pipelines
### North-Africa - Europe
The African continent is connected to the European continent by four main pipelines:

- Maghreb-Europe Gas Pipeline (Hassi R'Mel - Cordoba)
- Medgaz Pipeline (Hassi R'Mel - Almeria)
- Greenstream Pipeline (Mellitah - Sicily)
- Trans-Mediterranean Pipeline (Hassi R'Mel - Sicily)

As from 1 November 2021 however, the Maghreb-Europe Gas Pipeline is no longer operational, due to escalating rivarly between Algeria and Morocco (https://www.cidob.org/es/publicaciones/serie_de_publicacion/notes_internacionals_cidob/260/escalating_rivalry_between_algeria_and_morocco_closes_the_maghreb_europe_pipeline).

### Russia - Balticum - Europe
Through the Baltic countries, the Russia is connected to the European continent by three main pipelines:

- Valdai-Pskov-Riga Gas Pipeline
- Minsk-Kaliningrad Interconnection Gas Pipeline
- Kohtla-Järve-Leningrad Gas Pipeline

### Russia - Belarus - Europe
- Yamal-Europe Gas Pipeline
- Torzhok-Minsk-Ivatsevichy Gas Pipeline

### Russia - Ukraine - Europe
Major pipelines from Russia connected natural gas fields in Western Siberia to export markets in Western Europe through Ukraine. This can be split into a handful of entry and exit points.

#### Entry points
- Torzhok-Smolensk-Mazyr-Dolyna Gas Pipeline, through Belarus
- Dashava-Kiev-Bryansk-Moscow Gas Pipeline, through Bryansk
- Tula-Shostka-Kiev Gas Pipeline, through Bryansk
- Yelets-Kursk-Kyiv Gas Pipeline, through Kursk
- Urengoy-Pomary-Uzhgorod Gas Pipeline, through Kursk
- Progress Gas Pipeline, through Kursk
- Yelets-Kursk-Dykanka Gas Pipeline, through Kursk/Belgorod
- Yelets-Kremenchuk–Kryvyi Rih Gas Pipeline, through Belgorod
- Urengoy-Novopskov Gas Pipeline, through Voronezh
- Petrovsk-Novopskov Gas Pipeline, through Voronezh
- Soyuz Gas Pipeline, through Voronezh
- Orenburg-Novopskov Gas Pipeline, through Volgograd

#### Exit points
- Anan'iv-Tiraspol-Izmail Gas Pipeline, to Romania
- Urengoy-Pomary-Uzhgorod Gas Pipeline, to Slovakia (Part of the Brotherhood Gas Pipeline Network)
- Soyuz Gas Pipeline, to Slovakia (Part of the Brotherhood Gas Pipeline Network)
- Progress Gas Pipeline, to Slovakia (Part of the Brotherhood Gas Pipeline Network)
- Torzhok-Smolensk-Mazyr-Dolyna Gas Pipeline (Not sure if this transports gas to Europe) (Part of the Brotherhood Gas Pipeline Network)
- Kyiv-Western Border of Ukraine Gas Pipeline, to Poland
- Kyiv–Western Border Pipeline, to Slovakia
- Shebelinka-Dnipropetrovsk–Kryvyi Rih–Rozdilna-Izmail Gas Pipeline, to Romania



#### Intra-Ukraine

### (Russia) - Türkiye - Europe

- TurkStream 1 Gas Pipeline, to Greece (Ipsala)
- TurkStream 2 Gas Pipeline, to Bulgaria (Malkoçlar)
- Turkey-Greece-Italy Interconnector Gas Pipeline, to Greece (Komotini)
- Trans-Anatolian Gas Pipeline, to Greece (Ipsala)
- Sarikali-Karacabey Gas Pipeline (Not sure if thi transports gas to Europe)
- Trans-Balkan Gas Pipeline, to Bulgaria (Sofia) and to Ukraine (connecting with the Shebelinka-Dnipropetrovsk–Kryvyi Rih–Rozdilna-Izmail Gas Pipeline)
- Murati-Silivri Pipeline
- Russian Federation – Turkey Natural Gas Main Transmission Line (Not sure if operational)

In [30]:
relevant_pipelines_df

,ProjectID,Fuel,StartCountry,EndCountry,Countries,Wiki,PipelineName,SegmentName,OtherEnglishNames,Status,...,EndSubRegion,FIDYear,OtherLanguagePrimaryPipelineName,OtherLanguageSegmentName,Cost,CostUnits,CostUSD,WKTFormat,LastUpdated,geometry
224,P0439,Gas,Libya,Italy,"Libya, Italy",https://www.gem.wiki/Greenstream_Pipeline,Greenstream Gas Pipeline,,"Melita - Gela Sicily Pipeline, Melita TransGas",Operating,...,Southern Europe,,خط غاز جرين ستريم، خط انابيب الغاز,,6600000000,USD,6600000000,"LINESTRING (12.113564 32.9141632, 12.1708807 3...",2022-07-20,"LINESTRING (12.11356 32.91416, 12.17088 33.016..."
238,P0453,Gas,Algeria,Spain,"Algeria, Morocco, Spain, Portugal",https://www.gem.wiki/Maghreb-Europe_Gas_Pipeline,Maghreb-Europe Gas Pipeline,,Europe-Maghreb Pipeline (EMPL),Mothballed,...,Southern Europe,,"خط أنابيب غازالمغرب العربي-أوروبا, بيدرو دوران...",,,,--,"LINESTRING (3.2514804 32.9358193, -0.3197216 3...",2023-09-22,"LINESTRING (3.25148 32.93582, -0.31972 33.3306..."
349,P0725,Gas,Algeria,Italy,"Algeria, Croatia, Tunisia, Italy",https://www.gem.wiki/Trans-Mediterranean_Gas_P...,Trans-Mediterranean Gas Pipeline,,Transmed,Operating,...,Southern Europe,,"خط أنابيب ترانسميد, أنبوب غاز انريكو ماتي",,6250000000,USD,6250000000,"LINESTRING (3.250837 32.955169, 4.469612 34.17...",2022-07-08,"LINESTRING (3.25084 32.95517, 4.46961 34.17384..."
359,P0736,Gas,Russia,Türkiye,"Russia, Türkiye",https://www.gem.wiki/Blue_Stream_Gas_Pipeline,Blue Stream Gas Pipeline,,,Operating,...,Western Asia,,"Газопровод ""Голубой поток""",,3200000000,USD,3200000000,"LINESTRING (41.713627 45.368954, 41.715699 45....",2023-08-08,"LINESTRING (41.71363 45.36895, 41.71570 45.364..."
369,P0747,Gas,Türkiye,Greece,"Türkiye, Greece",https://www.gem.wiki/Interconnector_Turkey-Gre...,Turkey-Greece-Italy Interconnector Gas Pipelin...,,,Operating,...,Southern Europe,,,,,,--,"LINESTRING (28.3434944 40.2152939, 27.6128604 ...",2022-09-12,"LINESTRING (28.34349 40.21529, 27.61286 40.278..."
372,P0750,Gas,Belarus,Russia,"Belarus, Lithuania, Russia",https://www.gem.wiki/Minsk-Kaliningrad_Interco...,Minsk-Kaliningrad Interconnection Gas Pipeline,,,Operating,...,Eastern Europe,,Газопровод Минск — Вильнюс — Каунас — Калининград,,,,--,"LINESTRING (27.565267 53.902132, 25.592319 54....",2023-08-11,"LINESTRING (27.56527 53.90213, 25.59232 54.895..."
381,P0761,Gas,Russia,Ukraine,"Russia, Ukraine, Kazakhstan",https://www.gem.wiki/Soyuz_Gas_Pipeline,Soyuz Gas Pipeline,,,Mothballed,...,Eastern Europe,,Союз; also Оренбург - Западная граница СССР,,,,--,"LINESTRING (55.065 51.029, 51.402264 51.243895...",2023-08-25,"LINESTRING (55.06500 51.02900, 51.40226 51.243..."
383,P0763,Gas,Georgia,Türkiye,"Georgia, Türkiye",https://www.gem.wiki/Trans-Anatolian_Gas_Pipeline,Trans-Anatolian Gas Pipeline,,TANAP,Operating,...,Western Asia,,,,11000000000,USD,11000000000,"LINESTRING (26.3832 40.9632, 26.3856 40.9646, ...",2023-08-30,"LINESTRING (26.38320 40.96320, 26.38560 40.964..."
385,P0765,Gas,Russia,Türkiye,"Russia, Türkiye",https://www.gem.wiki/TurkStream_Gas_Pipeline,TurkStream Gas Pipeline,1,Turkish Stream 1 Gas Pipeline,Operating,...,Western Asia,,Газопровод Турецкий поток 1,,7800000000,USD,7800000000,"LINESTRING (37.3032745 44.8879545, 36.5430605 ...",2023-08-11,"LINESTRING (37.30327 44.88795, 36.54306 44.088..."
388,P0768,Gas,Russia,Ukraine,"Russia, Ukraine",https://www.gem.wiki/Urengoy-Pomary-Uzhgorod_G...,Urengoy-Pomary-Uzhgorod Gas Pipeline,,"Brotherhood Pipeline, West-Siberian Pipeline, ...",Operating,...,Eastern Europe,,Система газопроводов Уренгой - Помары - Ужгород,,,,--,"LINESTRING (76.51694 66.48333, 76.306915 66.43...",2023-08-09,"LINESTRING (76.51694 66.48333, 76.30692 66.438..."


## Adding pipelines from GEM dataset to SciGRID

In [31]:
# Read processed_data/IGGIN_Nodes_clean.csv
scigrid_nodes_df = pd.read_csv('processed_data/IGGIN_Nodes_cleaned.csv')

# Read processed_data/IGGIN_PipeSegments_clean.csv
scigrid_pipelines_df = pd.read_csv('processed_data/IGGIN_PipeSegments_cleaned.csv')


In [24]:
# I will now write a function. The function will take in a pipeline and a node. The pipeline will be from the relevant_pipelines_df and the node will be from the scigrid_nodes_df. The function will create a
# new pipeline in the scigrid_pipelines_df. This pipeline will be the same as the input pipeline, the onlye difference being that the end node will be the input node.

def create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, ProjectID, end_node_id):
    
    # Check that the columns of the scigrid_nodes_df are only: ['id', 'name', 'source_id', 'node_id', 'lat', 'long', 'country_code', 'comment']
    assert list(scigrid_nodes_df.columns) == ['id', 'name', 'source_id', 'node_id', 'lat', 'long', 'country_code', 'comment'], 'The columns of the scigrid_nodes_df are not as expected'

    # Check that the columns of the scigrid_pipelines_df are only: ['id', 'name', 'comment', 'diameter_mm', 'end_year', 'is_H_gas', 'is_bothDirection', 'lat_mean', 'length_km', 'long_mean', 'max_cap_M_m3_per_d', 'max_pressure_bar', 'num_compressor', 'start_year', 'waterDepth_m', 'from_node_id', 'to_node_id', 'from_country_code', 'to_country_code', 'from_lat', 'to_lat', 'from_long', 'to_long']
    assert list(scigrid_pipelines_df.columns) == ['id', 'name', 'comment', 'diameter_mm', 'end_year', 'is_H_gas', 'is_bothDirection', 'lat_mean', 'length_km', 'long_mean', 'max_cap_M_m3_per_d', 'max_pressure_bar', 'num_compressor', 'start_year', 'waterDepth_m', 'from_node_id', 'to_node_id', 'from_country_code', 'to_country_code', 'from_lat', 'to_lat', 'from_long', 'to_long'], 'The columns of the scigrid_pipelines_df are not as expected'

    # Check that the relevant_pipelines_df has the correct columns
    assert 'geometry' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the geometry column'
    assert 'CapacityBcm/y' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the CapacityBcm/y column'
    assert 'StartLocation' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the StartLocation column'
    assert 'StartCountry' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the StartCountry column'
    assert 'ProjectID' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the ProjectID column'
    assert 'PipelineName' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the PipelineName column'
    assert 'LengthKnownKm' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the LengthKnownKm column'
    assert 'StartYear1' in relevant_pipelines_df.columns, 'The relevant_pipelines_df does not have the StartYear1 column'

    # Check that the pipeline_name is in the relevant_pipelines_df
    assert ProjectID in relevant_pipelines_df['ProjectID'].values, 'The ProjectID is not in the relevant_pipelines_df'

    # Check that the node_id is in the scigrid_nodes_df
    assert end_node_id in scigrid_nodes_df['id'].values, 'The node_id is not in the scigrid_nodes_df'

    # Create a new node in the scigrid_nodes_df, corresponding to the start position of the pipeline
    
    # Set the id to the ProjectID of the pipeline
    start_node_id = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['ProjectID'].values[0]

    # Set the name to the StartLocation of the pipeline
    name = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['StartLocation'].values[0]

    # Set the source_id, and the node_id to the new_node_id
    source_id = start_node_id
    node_id = start_node_id

    # Set the lat and long to the lat and long of the first point in the geometry of the pipeline
    lat = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['geometry'].values[0].xy[1][0]
    long = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['geometry'].values[0].xy[0][0]

    # Set the country_code to the two letter country code of the StartCountry of the pipeline
    country_code = pycountry.countries.get(name=relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['StartCountry'].values[0]).alpha_2

    # Set the comment to : "This node was created to connect the GEM pipeline to the SciGRID network"
    comment = "This node was created to connect the GEM pipeline to the SciGRID network"

    # Create a new row in the scigrid_nodes_df with the above values, using concat
    new_node = pd.DataFrame([[start_node_id, name, source_id, node_id, lat, long, country_code, comment]], columns=scigrid_nodes_df.columns)
    scigrid_nodes_df = pd.concat([scigrid_nodes_df, new_node], ignore_index=True)

    # Create a new pipeline in the scigrid_pipelines_df, corresponding to the pipeline and the new node

    # Check if the pipeline has been created before, by checking if the ProjectID of the pipeline is in the scigrid_pipelines_df
    if start_node_id in scigrid_pipelines_df['from_node_id'].values:
        # If the pipeline has been created before, throw an error
        raise ValueError('The pipeline has already been created')

    # Set the id to the project_id of the pipeline
    pipeline_id = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['ProjectID'].values[0]

    # Set the name to the PipelineName of the pipeline
    name = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['ProjectID'].values[0]

    # Set the comment to "This pipeline was created to connect the GEM pipeline to the SciGRID network"
    comment = "This pipeline was created to connect the GEM pipeline to the SciGRID network"

    # Try to set the diameter_mm to the Diameter of the pipeline. Split the Diameter by the ",", strip the whitespace and take the first element
    try:
        diameter_mm = int(relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['Diameter'].values[0].split(',')[0].strip())
    except:
        # If the diameter_mm cannot be set, set it to 0
        diameter_mm = 0

    # As the diameter_mm is in inches, convert it to mm
    diameter_mm = diameter_mm * 25.4

    # Set the end_year to to 2050
    end_year = 2050

    # Set the is_H_gas to 1.0
    is_H_gas = 1.0

    # Set the is_bothDirection to 0.0
    is_bothDirection = 0.0

    # Set the lat_mean to the average of the lat of the start and end node
    lat_mean = (lat + scigrid_nodes_df[scigrid_nodes_df['id'] == end_node_id]['lat'].values[0]) / 2

    # Set the length_km to the LengthKnownKm of the pipeline
    try:
        length_km = float(relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['LengthKnownKm'].values[0])
    except:
        # If the length_km cannot be set, set it to 0
        print(f"The length of the pipeline {ProjectID} is not known. Setting the length to 0")
        length_km = 0

    # Set the long_mean to the average of the long of the start and end node
    long_mean = (long + scigrid_nodes_df[scigrid_nodes_df['id'] == end_node_id]['long'].values[0]) / 2

    # Set the max_cap_M_m3_per_d to the CapacityBcm/y of the pipeline
    try:
        max_cap_M_m3_per_d = float(relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['CapacityBcm/y'].values[0]) * 1e6 / 365
        
        # Convert from billion cubic meters per year to million cubic meters per day
        max_cap_M_m3_per_d = max_cap_M_m3_per_d * 1000 / 365

    except:
        # If the max_cap_M_m3_per_d cannot be set, set it to 0
        print(f"The capacity of the pipeline {ProjectID} is not known. Setting the capacity to 0")
        max_cap_M_m3_per_d = 0    

    # Set the max_pressure_bar to nan
    max_pressure_bar = float('nan')

    # Set the num_compressor to 0
    num_compressor = 0

    # Set the start_year to StartYear1
    start_year = relevant_pipelines_df[relevant_pipelines_df['ProjectID'] == ProjectID]['StartYear1'].values[0]

    # Set the waterDepth_m to nan
    waterDepth_m = float('nan')

    # Set the from_node_id to the start_node_id
    from_node_id = start_node_id

    # Set the to_node_id to the end_node_id
    to_node_id = end_node_id

    # Set the from_country_code to the country_code of the start node
    from_country_code = country_code

    # Set the to_country_code to the country_code of the end node
    to_country_code = scigrid_nodes_df[scigrid_nodes_df['id'] == end_node_id]['country_code'].values[0]

    # Set the from_lat to the lat of the start node
    from_lat = lat

    # Set the to_lat to the lat of the end node
    to_lat = scigrid_nodes_df[scigrid_nodes_df['id'] == end_node_id]['lat'].values[0]

    # Set the from_long to the long of the start node
    from_long = long

    # Set the to_long to the long of the end node
    to_long = scigrid_nodes_df[scigrid_nodes_df['id'] == end_node_id]['long'].values[0]

    # Create a new row in the scigrid_pipelines_df with the above values, using concat
    new_pipeline = pd.DataFrame([[pipeline_id, name, comment, diameter_mm, end_year, is_H_gas, is_bothDirection, lat_mean, length_km, long_mean, max_cap_M_m3_per_d, max_pressure_bar, num_compressor, start_year, waterDepth_m, from_node_id, to_node_id, from_country_code, to_country_code, from_lat, to_lat, from_long, to_long]], columns=scigrid_pipelines_df.columns)
    scigrid_pipelines_df = pd.concat([scigrid_pipelines_df, new_pipeline], ignore_index=True)

    return scigrid_nodes_df, scigrid_pipelines_df

In [25]:
# Trans-Mediterranean Gas Pipeline
scigrid_nodes_df, scigrid_pipelines_df = create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, "P0725", "INET_N_935")

# Greenstream Gas Pipeline
scigrid_nodes_df, scigrid_pipelines_df = create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, "P0439", "INET_N_530")

# Medgaz Gas Pipeline
scigrid_nodes_df, scigrid_pipelines_df = create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, "P2055", "INET_N_38")
scigrid_nodes_df, scigrid_pipelines_df = create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, "P2056", "INET_N_38")

# Maghreb-Europe Gas Pipeline
scigrid_nodes_df, scigrid_pipelines_df = create_pipeline(scigrid_nodes_df, relevant_pipelines_df, scigrid_pipelines_df, "P0453", "INET_N_1650")


AttributeError: 'NoneType' object has no attribute 'xy'

In [ ]:
# Plot the scigrid_nodes_df on a map
m = fl.Map(location=[50, 10], zoom_start=4)

# Create a feature group for scigrid nodes
nodes_layer = fl.FeatureGroup(name='SciGRID Nodes')

# Add the scigrid nodes layer to the map using the lat and long columns
for idx, row in scigrid_nodes_df.iterrows():
    fl.Marker(location=[row['lat'], row['long']], popup=row['name']).add_to(nodes_layer)

# Add the scigrid pipelines layer to the map using the from_lat, from_long, to_lat and to_long columns
pipelines_layer = fl.FeatureGroup(name='SciGRID Pipelines')
for idx, row in scigrid_pipelines_df.iterrows():
    fl.PolyLine([(row['from_lat'], row['from_long']), (row['to_lat'], row['to_long'])], color="blue", weight=2.5, opacity=1).add_to(pipelines_layer)

# Add the feature groups to the map
nodes_layer.add_to(m)
pipelines_layer.add_to(m)

# Add layer control to the map
fl.LayerControl().add_to(m)

m